In [3]:
import networkx as nx # the main libary we will use
from networkx.algorithms import bipartite
from networkx.algorithms import community
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
%matplotlib inline 
import statistics
import networkx.algorithms.community
from datetime import datetime, timedelta
from datetime import date
import calendar

from networkx.algorithms import community
from networkx.algorithms import bipartite
from networkx.algorithms import centrality as ce
from networkx.algorithms import community as c

### Reading the data

In [4]:
file= 'data/Investments.xlsx'
df = pd.read_excel(file)

#### Preprocess

In [ ]:
df = df[['company_name','company_market','investor_name','funded_year','raised_amount_usd']]
df = df.dropna()
df = df[df.raised_amount_usd != 0]

### Data exploration

In [8]:
print(f"Number of unique investors: {df['investor_name'].nunique()}")
print(f"Number of unique companies: {df['company_name'].nunique()}")
print(f"Number of unique company markets: {df['company_market'].nunique()}")

# df["Route"] = df["Start Centroid Location"] + df["End Centroid Location"]
# print("The number of unique route in the data is " , df["Route"].nunique())

Number of unique investors: 22037
Number of unique companies: 32284
Number of unique company markets: 712


In [ ]:
# Reading 

### Building the network

In [9]:
G_weighted_directed = nx.from_pandas_edgelist(df=df,source='investor_name',target='company_name',edge_attr=True,create_using=nx.DiGraph())
attrs_n={}
nodes = pd.DataFrame(list(G_weighted_directed.nodes))

In [12]:
# degree = dict(G_weighted_directed.in_degree)
# print(degree)